# Qiskit Runtime

Qiskit Runtime is a new architecture offered by IBM Quantum that significantly reduces waiting time during 
computational iterations. You can execute your experiments near the quantum hardware, without 
the interactions of multiple layers of classical and quantum hardware slowing it down.

Using Qiskit Runtime, for example, a research team at IBM Quantum was able to achieve 120x speed 
up in their lithium hydride simulation (link to come). 

Qiskit Runtime allows authorized users to upload their Qiskit quantum programs for themselves or 
others to use. A Qiskit quantum program, also called a Qiskit runtime program, is a piece of Python code that takes certain inputs, performs
quantum and maybe classical computation, and returns the processing results. The same or other
authorized users can then invoke these quantum programs by simply passing in the required input parameters.

<div class="alert alert-block alert-info">
<b>Note:</b> Qiskit Runtime is only available to select IBM Quantum providers. You can use the `has_service()` method to check if a provider has access:
</div>

In [1]:
from qiskit import IBMQ

IBMQ.load_account()
provider = IBMQ.get_provider(project='qiskit-runtime')  # Change this to your provider.
can_use_runtime = provider.has_service('runtime')

If you don't have an IBM Quantum account, you can sign up for one on the [IBM Quantum](https://quantum-computing.ibm.com/) page.

## Listing programs <a name='listing_program'>

The `provider.runtime` object is an instance of the [`IBMRuntimeService`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.IBMRuntimeService.html#qiskit.providers.ibmq.runtime.IBMRuntimeService) class and serves as the main entry point to using the runtime service. It has three methods that can be used to find metadata of available programs:
- `pprint_programs()`: pretty prints metadata of all available programs
- `programs()`: returns a list of `RuntimeProgram` instances
- `program()`: returns a single `RuntimeProgram` instance

The metadata of a runtime program includes its ID, name, description, version, input parameters, return values, interim results, maximum execution time, and backend requirements. Maximum execution time is the maximum amount of time, in seconds, a program can run before being forcibly terminated.

To print the metadata of all available programs:

In [2]:
provider.runtime.pprint_programs()

circuit-runner:
  Name: circuit-runner
  Description: A runtime program that takes one or more circuits, compiles them, executes them, and optionally applies measurement error mitigation.
  Version: 1
  Creation date: 2021-05-07T00:17:07Z
  Max execution time: 1800
  Input parameters:
    - circuits:
      Description: A circuit or a list of circuits.
      Type: A QuantumCircuit or a list of QuantumCircuits.
      Required: True
    - shots:
      Description: Number of repetitions of each circuit, for sampling. Default: 1024.
      Type: int
      Required: False
    - initial_layout:
      Description: Initial position of virtual qubits on physical qubits.
      Type: dict or list
      Required: False
    - layout_method:
      Description: Name of layout selection pass ('trivial', 'dense', 'noise_adaptive', 'sabre')
      Type: string
      Required: False
    - routing_method:
      Description: Name of routing pass ('basic', 'lookahead', 'stochastic', 'sabre').
      Type: strin

To print the metadata of the program `runtime-simple`:

In [3]:
program = provider.runtime.program('runtime-simple')
print(program)

runtime-simple:
  Name: runtime-simple
  Description: Simple runtime program used for testing.
  Version: 1
  Creation date: 2021-05-05T23:30:21Z
  Max execution time: 300
  Input parameters:
    - iterations:
      Description: Number of iterations to run. Each iteration generates and runs a random circuit.
      Type: int
      Required: True
  Interim results:
    - iteration:
      Description: Iteration number.
      Type: int
    - counts:
      Description: Histogram data of the circuit result.
      Type: dict
  Returns:
    - -:
      Description: A string that says 'All done!'.
      Type: string


As you can see from above, the program `runtime-simple` is a simple program that has only 1 input parameter `iterations`, which indicates how many iterations to run. For each iteration it generates and runs a random 5-qubit circuit and returns the counts as well as the iteration number as the interim results. When the program finishes, it returns the sentence `All done!`. This program can only run for 300 seconds (5 minutes), and requires a backend that has at least 5 qubits.

## Invoking a runtime program <a name='invoking_program'>

You can use the [`IBMRuntimeService.run()`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.IBMRuntimeService.html#qiskit.providers.ibmq.runtime.IBMRuntimeService.run) method to invoke a runtime program. This method takes the following parameters:

- `program_id`: ID of the program to run
- `inputs`: Program input parameters. These input values are passed to the runtime program.
- `options`: Runtime options. These options control the execution environment. Currently the only available option is `backend_name`, which is required.
- `callback`: Callback function to be invoked for any interim results. The callback function will receive 2 positional parameters: job ID and interim result.
- `result_decoder`: Optional class used to decode job result.

Before we run a quantum program, we may want to define a callback function that would process interim results, which are intermediate data provided by a program while its still running. 

As we saw earlier, the metadata of `runtime-simple` says that its interim results are the iteration number and the counts of the randomly generated circuit. Here we define a simple callback function that just prints these interim results:

In [4]:
def interim_result_callback(job_id, interim_result):
    print(f"interim result: {interim_result}")

To run the `runtime-simple` program with 3 iterations on `ibmq_montreal`:

In [5]:
# backend = provider.get_backend('ibmq_montreal')
runtime_inputs = {
    'iterations': 3
}
options = {'backend_name': backend.name()}
job = provider.runtime.run(program_id="runtime-simple",
                           options=options,
                           inputs=runtime_inputs,
                           callback=interim_result_callback
                          )

interim result: {'iteration': 0, 'counts': {'00000': 30, '00001': 47, '10000': 32, '10001': 35, '10010': 22, '10011': 20, '10100': 25, '10101': 34, '10110': 28, '10111': 23, '11000': 30, '11001': 25, '11010': 14, '11011': 25, '11100': 34, '11101': 40, '11110': 45, '11111': 34, '00010': 40, '00011': 18, '00100': 27, '00101': 39, '00110': 36, '00111': 34, '01000': 28, '01001': 31, '01010': 27, '01011': 34, '01100': 40, '01101': 52, '01110': 42, '01111': 33}}
interim result: {'iteration': 1, 'counts': {'00000': 1, '00001': 6, '10000': 4, '10010': 32, '10011': 2, '10100': 26, '10101': 14, '10110': 147, '10111': 24, '11000': 11, '11001': 3, '11010': 61, '11011': 13, '11100': 69, '11101': 40, '11110': 222, '11111': 47, '00010': 8, '00011': 3, '00100': 10, '00101': 6, '00110': 70, '00111': 7, '01000': 9, '01001': 7, '01010': 22, '01011': 4, '01100': 27, '01101': 8, '01110': 101, '01111': 20}}
interim result: {'iteration': 2, 'counts': {'00000': 5, '00001': 51, '10000': 13, '10001': 17, '10010

The `run()` method returns a [`RuntimeJob`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.RuntimeJob.html#qiskit.providers.ibmq.runtime.RuntimeJob) instace, which is similar to the `Job` instance returned by regular `backend.run()`. `RuntimeJob` supports the following methods:

- `status()`: Return job status.
- `result()`: Wait for the job to finish and return the final result.
- `cancel()`: Cancel the job.
- `wait_for_final_state()`: Wait for the job to finish.
- `stream_results()`: Stream interim results. This can be used to start streaming the interim results if a `callback` function was not passed to the `run()` method.
- `job_id()`: Return the job ID.
- `backend()`: Return the backend where the job is run.

In [6]:
print(job.job_id())

c28abobi47qvbj8sa3b0


To get the final result of the job we just submitted:

In [7]:
result = job.result()
result

'All done!'

## Deleting a job

You can use the [`IBMRuntimeService.delete_job()`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.IBMRuntimeService.html#qiskit.providers.ibmq.runtime.IBMRuntimeService.delete_job) method to delete a job. You can only delete your own jobs, and this action cannot be reversed. 

In [8]:
import qiskit.tools.jupyter
%qiskit_version_table